In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

In [3]:
import time

In [4]:
import importlib

In [35]:
try:
    importlib.reload(functions)
except:
    import functions

In [36]:
try:
    importlib.reload(multiprocessing)
except:
    import multiprocessing

In [37]:
p= multiprocessing.Pool(processes = 8)

In [8]:
prices = pd.read_pickle("parsed_data/prices")

In [9]:
derived_shareprices = pd.read_pickle("parsed_data/derived_shareprices")

In [10]:
Tickers = np.unique(prices.index)

In [11]:
Date = prices.loc["A"]['Date']

In [12]:
technical_features = pd.read_pickle("parsed_data/technical_features")

In [9]:
technical_features = pd.DataFrame(prices["Date"], prices.index)

In [10]:
technical_features["Close"] = prices["Close"]

In [248]:
technical_features["ABAND_high"] = prices["High"] * (1 + 4 * (prices["High"] - prices['Low']) / (prices["High"] + prices["Low"]))

In [249]:
technical_features["ABAND_low"] = prices["Low"] * (1 - 4 * (prices["High"] - prices['Low']) / (prices["High"] + prices["Low"]))

In [250]:
"""
for indicator
"""

for window in [10, 20, 40, 80, 160]:
    start_time = time.time()
    to_append3 = np.array([])
    for i in Tickers:
        to_append =  technical_features.loc[i]["ABAND_high"].rolling(window).mean()
        to_append2 = technical_features.loc[i]["ABAND_low"].rolling(window).mean()

        to_append3 = np.append(to_append3, ( prices.loc[i]['Close'] - ((to_append + to_append2) / 2 ) ) / (to_append - to_append2) )


    technical_features["ABAND_indicator_"+str(window)] = to_append3 

    print(time.time() - start_time)

11.77737545967102
11.775643348693848
11.610604286193848
11.669618606567383
11.67561936378479


In [251]:
"""
for signal
"""
for window in [10, 20, 40, 80, 160]:
    
    to_append3 = np.array([])
    start_time = time.time()
    for i in Tickers:
        to_append =  technical_features.loc[i]["ABAND_high"].rolling(window).mean()
        to_append2 = technical_features.loc[i]["ABAND_low"].rolling(window).mean()

        distance = 0.1 * np.abs(to_append - to_append2)

        array = np.zeros(len(to_append))

        where  = np.where(prices.loc[i]["Close"] > (to_append - distance))[0]

        array[where] = 1

        where = np.where(prices.loc[i]["Close"] < to_append2 + distance  )[0]

        array[where] = -1



        to_append3 = np.append(to_append3, array)

    technical_features["ABAND_signal_"+str(window)] = to_append3
    print(time.time() - start_time)

12.737514972686768
12.547815322875977
12.366773843765259
12.325765609741211
12.430789947509766


In [252]:
technical_features = technical_features.drop(['ABAND_high', 'ABAND_low' ], axis = 1)

In [19]:
start_time = time.time()
window = 40
shift_val = 10

to_append = np.array([])

for i in Tickers:
    thing = ( ( (prices.loc[i]["Close"] - (prices.loc[i]["Low"].rolling(window).min())  ) - ( (prices.loc[i]['High'].rolling(window).max()) - prices.loc[i]['Close']) ) / ( (prices.loc[i]['High'].rolling(window).max()) - (prices.loc[i]["Low"].rolling(window).min()) ) ) * (prices.loc[i]['Volume'].rolling(window).sum())
    to_append = np.append(to_append, thing)
    
technical_features["MFV_"+str(window)] = to_append



to_append = np.array([])
for i in Tickers:
    first_notna = technical_features.loc[i]["MFV_"+str(window)][np.where(technical_features.loc[i]["MFV_"+str(window)].notna())[0][0]]
    thing = technical_features.loc[i]["MFV_"+str(window)].cumsum() - first_notna
    to_append = np.append(to_append, thing )
    
technical_features["MFV_cumulative_"+str(window)]  = to_append

to_append = np.array([])

for i in Tickers:
    thing = ( technical_features.loc[i]["MFV_cumulative_"+str(window)]  -  technical_features.loc[i]["MFV_cumulative_"+str(window)].shift(shift_val) ) / shift_val
    to_append = np.append(to_append, thing)
    
technical_features["MFV_"+str(window) +"_" + str(shift_val) + "_indicator"] = to_append

to_append = np.array([])
for i in Tickers:
    where = np.where(technical_features.loc[i]["MFV_"+str(window)+"_" + str(shift_val) + "_indicator"].apply(np.sign).rolling(2).mean() == 0)[0]
    thing = np.zeros(len(technical_features.loc[i]["MFV_"+str(window)+"_" + str(shift_val) + "_indicator"]))
    for j in where:
        if np.sign(technical_features.loc[i]["MFV_"+str(window)+"_" + str(shift_val) + "_indicator"].iloc[j]) == 1:
            thing[j] = 1
        elif np.sign(technical_features.loc[i]["MFV_"+str(window)+"_" + str(shift_val) + "_indicator"].iloc[j]) == -1:
            thing[j] = -1
    to_append = np.append(to_append, thing)
    

technical_features["MFV_" + str(window) + "_" + str(shift_val) + "_indicator_sign_change"] = to_append

print(time.time() - start_time)


technical_features = technical_features.drop(["MFV_"+str(window), "MFV_cumulative_"+str(window), "MFV_"+str(window) +"_" + str(shift_val) + "_indicator"], axis = 1 )



55.437429428100586


In [ ]:
"""
notes on MFV:
MFV 40 seems to be sweet spot but have not tried any higher, because the higher it is the more time lag
The indicator is just the derivative and period of 10 seems to be best

Could we have longer windows: 80? using exponential weughted means?

"""

In [20]:
for span_short, span_long in [[10,20],[10, 40]]:
    
    start_time = time.time()


    to_append = np.array([])

    for i in Tickers:
        thing = (prices.loc[i]["Close"].ewm(span = span_short).mean() - prices.loc[i]["Close"].ewm(span = span_long).mean() ) / prices.loc[i]["Close"].ewm(span = span_long).mean()
        to_append = np.append(to_append, thing)

    technical_features["APO_"+str(span_short) + "_"+ str(span_long)] = to_append
    
    print(time.time() - start_time)
    


8.277857303619385
8.656941890716553


In [250]:
Tickers_split = np.array_split(Tickers, 10)

for window in [10, 20, 40, 80, 120, 160]:
    
    start_time = time.time()
    
    tasks = zip([window] * 10, Tickers_split)
    
    output = p.starmap(functions.aroon_indicator, tasks)
    
    to_append = np.array([])

    for i in output:
        to_append = np.append(to_append, i)
        
    technical_features["AROON_"+str(window)] = to_append
    
    print(time.time() - start_time)
        
    
        
    
    
    

96.48182034492493
94.96630811691284
95.63245820999146
92.83883142471313
90.90139651298523
89.5450918674469


In [263]:
technical_features.to_pickle("parsed_data/technical_features")

In [15]:
for window in [10, 20, 40, 80, 120, 160]:
    
    start_time = time.time()
    
    to_append = np.array([])

    for i in Tickers:
        moving_average = prices.loc[i]["Close"].ewm(span = window).mean()

        BOL_indicator =  ( prices.loc[i]['Close'] - moving_average ) / (4 * prices.loc[i]["Close"].rolling(window).std() )

        to_append = np.append(to_append, BOL_indicator)
        
    technical_features["BOL_indicator_"+str(window)] = to_append
    
    print(time.time() - start_time)

9.765192031860352
9.827205657958984
9.827204704284668
9.77519416809082
9.726194620132446


In [17]:
for window in [10, 20, 40, 80, 120, 160]: 
    to_append = np.array([])
    
    start_time = time.time()
    
    for i in Tickers:
        test = prices.loc[i]["Close"] - prices.loc[i]['Close'].shift(1) 
        testpos = test * (test > 0)
        testneg = test * (test < 0)

        possum = testpos.rolling(window).sum()
        negsum = testneg.rolling(window).sum().abs()

        to_append = np.append(to_append,  ( (possum - negsum) / (possum + negsum) ) * 100 )
        
    technical_features["CMO_"+str(window)] = to_append
    
    
    print(time.time() - start_time)
    

10.604379415512085
10.781419038772583
11.817651510238647
10.972461700439453
10.727407455444336


In [120]:
"""
average directional index

"""
for window in [1, 4, 16]:
    start_time = time.time()
    
    ADX_5_array = np.array([])
    ADX_11_array = np.array([])
    ADX_23_array = np.array([])
    
    
    for i in Tickers:
        
        plusDMinit = prices.loc[i]["High"].rolling(window).max() - prices.loc[i]["High"].shift(window).rolling(window).max()
        minusDMinit = prices.loc[i]["Low"].shift(window).rolling(window).min() - prices.loc[i]["Low"].rolling(window).min()
        
        plusDM = plusDMinit.copy()
        minusDM = minusDMinit.copy()
        
        plusDM[np.where(plusDMinit < minusDMinit)[0]] = 0
        minusDM[np.where(plusDMinit > minusDMinit)[0]] = 0
        
        plusDM[np.where((np.sign(plusDMinit) + np.sign(minusDMinit)) == -2)[0]] = 0
        minusDM[np.where((np.sign(plusDMinit) + np.sign(minusDMinit)) == -2)[0]] = 0
        
        
        a = prices.loc[i]["High"].rolling(window).max() - prices.loc[i]["Low"].rolling(window).min()
        b = prices.loc[i]["High"].rolling(window).max() - prices.loc[i]["Close"].shift(window)
        c = prices.loc[i]["Close"].shift(window) - prices.loc[i]["Low"].rolling(window).min()
        
        TR = np.amax([a,b,c], axis = 0)
        
        for window2 in [3*2 - 1, 6 * 2 -1, 12 * 2 -1]:
            plusDM_smoothed = pd.DataFrame(plusDM).ewm(span = window2).mean().values
            minusDM_smoothed = pd.DataFrame(minusDM).ewm(span = window2).mean().values
            TR_smoothed = pd.DataFrame(TR).ewm(span = window2).mean().values
            
            plusDI = (plusDM_smoothed / TR_smoothed) * 100
            minusDI = (minusDM_smoothed  / TR_smoothed) * 100
            
            DiAbsDiff = np.abs(plusDI - minusDI)
            
            DiSum = plusDI + minusDI
            
            DX =  (DiAbsDiff / DiSum) * 100
            
            if window2 == 3*2 - 1:
                            
                ADX_5 = pd.DataFrame(DX).ewm(span = window2).mean().values
                
            elif window2 == 6 * 2 -1:
                
                ADX_11 = pd.DataFrame(DX).ewm(span = window2).mean().values
                
            else:
                
                ADX_23 = pd.DataFrame(DX).ewm(span = window2).mean().values
                
        
        ADX_5_array = np.append(ADX_5_array, ADX_5)  
        ADX_11_array = np.append(ADX_11_array, ADX_11)
        ADX_23_array = np.append(ADX_23_array, ADX_23)
        
        
    technical_features["ADX_" + str(window) + "_5"] = ADX_5_array
    technical_features["ADX_" + str(window) + "_11"] = ADX_11_array
    technical_features["ADX_" + str(window) + "_23"] = ADX_23_array
    
    print(time.time() - start_time)
            

    

53.710922956466675
53.68004512786865
57.15839600563049


In [122]:
for window in [10, 20, 40, 80, 120, 160]:
    to_append = np.array([])
    for i in Tickers:
        average = prices.loc[i]["Close"].ewm(span = window).mean()
        thing =  ( 2 * average ) - ( average.ewm(span = window).mean() ) 
        to_append = np.append(to_append, (prices.loc[i]['Close'].rolling(3).mean() - thing) / thing )
        
    technical_features["DEMA_"+str(window)] = to_append

In [124]:
technical_features.to_pickle("parsed_data/technical_features")

In [13]:
technical_features.columns

Index(['Date', 'Close', 'ABAND_indicator_10', 'ABAND_indicator_20',
       'ABAND_indicator_40', 'ABAND_signal_10', 'ABAND_signal_20',
       'ABAND_signal_40', 'MFV_40_10_indicator_sign_change', 'APO_10_20',
       'APO_10_40', 'AROON_10', 'AROON_20', 'AROON_40', 'AROON_80',
       'AROON_120', 'AROON_160', 'BOL_indicator_10', 'BOL_indicator_20',
       'BOL_indicator_40', 'BOL_indicator_80', 'BOL_indicator_120',
       'BOL_indicator_160', 'CMO_10', 'CMO_20', 'CMO_40', 'CMO_80', 'CMO_120',
       'CMO_160', 'ADX_1_5', 'ADX_1_11', 'ADX_1_23', 'ADX_4_5', 'ADX_4_11',
       'ADX_4_23', 'ADX_16_5', 'ADX_16_11', 'ADX_16_23', 'DEMA_10', 'DEMA_20',
       'DEMA_40', 'DEMA_80', 'DEMA_120', 'DEMA_160'],
      dtype='object')

In [14]:
for window in [10, 20, 40, 80, 160]:
    to_append = np.array([])
    start_time = time.time()
    for i in Tickers:
        ATR = ( prices.loc[i]['Close'] + prices.loc[i]["Low"] + prices.loc[i]["High"] ) /3
        thing = ( ( ATR.rolling(window).mean() - ATR.shift(window).rolling(window).mean() ) / ATR.shift(window).rolling(window).mean() ) * 100
        to_append = np.append(to_append, thing)
        
    technical_features["ATR_" + str(window)] = to_append
    
    print(time.time() - start_time)

11.317264556884766
11.089000225067139
11.091889142990112
11.286532402038574
11.055382013320923


In [15]:
for window in [10, 20, 40, 80, 160]:
    to_append = np.array([])
    start_time = time.time()
    for i in Tickers:
        ATR = ( prices.loc[i]['Close'] + prices.loc[i]["Low"] + prices.loc[i]["High"] ) /3
        ATR_signed = np.sign( ATR - ATR.shift(1) )

        Money_flow = ( ( prices.loc[i]['Close'] + prices.loc[i]["Low"] + prices.loc[i]["High"] ) /3 ) * prices.loc[i]['Volume']
        Money_flow_pos = Money_flow.copy()
        Money_flow_neg = Money_flow.copy()

        Money_flow_pos.iloc[np.where(ATR_signed == -1)[0] ] = 0
        Money_flow_neg.iloc[np.where(ATR_signed == 1)[0] ] = 0

        pos_sum = Money_flow_pos.rolling(window).sum()
        neg_sum = Money_flow_neg.rolling(window).sum()

        thing = 100 - (100 / (1 + (pos_sum / neg_sum )))
        to_append = np.append(to_append, thing)

    
    technical_features["MFI_"+str(window)] = to_append
    
    print(time.time() - start_time)





14.048151969909668
13.968206882476807
14.200185537338257
14.358992576599121
14.158177614212036


In [16]:
for window in [10, 20, 40, 80, 160]:
    to_append = np.array([])
    start_time = time.time()
    for i in Tickers:
        thing = np.sign(prices.loc[i]['Close'] - prices.loc[i]["Close"].shift(1) )
        thing2 = thing * prices.loc[i]['Volume']
        thing3 = thing2.cumsum()

        thing4 = thing3.ewm(span = 3).mean()
        thing5 = (thing4 - thing4.shift(window) )/ thing4.shift(window) * 100
        to_append = np.append(to_append, thing5)
     
    technical_features["OBV_"+str(window)] = to_append
    
    print(time.time() - start_time)
    

10.822017669677734
10.745412111282349
10.648865938186646
10.597341775894165
11.081487655639648


In [17]:
for k in [10, 20, 40, 70, 100, 130, 160, 200, 240, 280, 320, 360]:
    to_append = np.array([])
    for i in Tickers:
        thing = ( prices.loc[i]['Close'] - prices.loc[i]['Close'].shift(k) ) /  prices.loc[i]['Close'].shift(k) * 100
        to_append = np.append(to_append, thing)

    technical_features["close_0/Close_"+str(k)] = to_append

In [18]:
for q in [10, 20, 40, 80, 120, 160, 240, 320]:
    
    start_time = time.time()
    for j in [20, 40, 80, 120, 160, 240, 320, 360]:
        
        to_append = np.array([])
        if q < j:
            for i in Tickers:
                thing = ( prices.loc[i]['Close'].shift(q) - prices.loc[i]['Close'].shift(j) ) /  prices.loc[i]['Close'].shift(j) * 100
                to_append = np.append(to_append, thing)

            technical_features["close_" + str(q) + "/Close_" + str(j)] = to_append
            
    print(time.time() - start_time)

84.98471188545227
73.91354894638062
65.96795916557312
55.25373148918152
42.82905387878418
32.077064514160156
21.756408214569092
11.019472599029541


In [160]:

to_append = np.array([])
for i in Tickers:
    start_time - time.time()
    to_append = np.append(to_append, ( prices.loc[i]['Close'] - prices.loc[i]['Close'].shift(1) ) /   prices.loc[i]['Close'].shift(1) * 100 )
    
prices["daily_return"] = to_append
print(time.time() - start_time)

3182.436245918274


In [20]:
for k in [10, 20, 40, 80, 120, 160, 240, 320, 360]:
    to_append = np.array([])
    start_time = time.time()
    for i in Tickers:
        thing = prices["daily_return"].loc[i].rolling(k).std()
        to_append = np.append(to_append, thing)
        
    print(time.time() - start_time)

    technical_features["volatility_" + str(k)] = to_append

9.704909801483154
7.31364107131958
9.248795509338379
9.382172584533691
9.726182460784912
8.387778997421265
7.261355400085449
8.23927903175354
9.197063207626343


In [21]:
for k in [10, 20, 40, 80, 120, 160, 240, 320, 360]:
    start_time = time.time()
    to_append = np.array([])
    for i in Tickers:
        thing = prices["Volume"].loc[i].rolling(k).std()
        to_append = np.append(to_append, thing)
    print(time.time() - start_time)
    technical_features["volstd_" + str(k)] = to_append

8.912506103515625
9.090039730072021
8.448895931243896
9.023053884506226
8.224845170974731
8.400355100631714
8.025557279586792
8.058807611465454
9.058649063110352


In [22]:
for k in [10, 20, 40, 80, 160]:
    to_append = np.array([])
    start_time = time.time()
    for i in Tickers:
        thing = -100 * ( (prices["High"].loc[i].rolling(k).max() - prices.loc[i]["Close"]) / (prices["High"].loc[i].rolling(k).max()  - prices["Low"].loc[i].rolling(k).min()  ) )
        to_append = np.append(to_append, thing)
        
    technical_features["WillR_" + str(k)] = to_append
    print(time.time() - start_time)

11.569207906723022
10.66939401626587
10.611380338668823
10.401333808898926
10.448344230651855


In [23]:
technical_features.to_pickle("parsed_data/technical_features")

In [41]:
BP = np.array([])
TR = np.array([])
for i in Tickers:
        
    BP = np.append(BP, (prices.loc[i]["Close"] - np.minimum(prices.loc[i]["Low"].values, prices.loc[i]["Close"].shift(1).values)))
    TR = np.append(TR, (np.maximum(prices.loc[i]["High"].values, prices.loc[i]["Close"].shift(1).values) - np.minimum(prices.loc[i]["Low"].values, prices.loc[i]["Close"].shift(1).values)))

In [44]:
technical_features["BP"] = BP
technical_features["TR"] = TR

In [154]:
np.geterr()

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [153]:
np.seterr(divide = "ignore", invalid = "ignore")

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [155]:
"""
first bit of ultimate oscillator

"""
start_time = time.time()
current_time = start_time
for window in [28, 56, 112, 224]:
    
    start_time = time.time()
    to_append = np.array([])
    
    for i in Tickers:
            
        BP = technical_features.loc[i]["BP"]
        TR = technical_features.loc[i]["TR"]
              
        average_short = BP.rolling(int(window/4)).sum().values / TR.rolling(int(window/4)).sum().values
        average_mid = BP.rolling(int(window/2)).sum().values / TR.rolling(int(window/2)).sum().values
        average_long = BP.rolling(int(window)).sum().values / TR.rolling(int(window)).sum().values
        
        ULTOSC = 100 *( (4 * average_short) + (2 * average_mid) + average_long) / 7
              
        to_append = np.append(to_append, ULTOSC)
        
    technical_features["ULTOSC_"+str(window)] = to_append
        
    print(time.time() - start_time)
    

9.97123670578003
10.000244140625
10.116268634796143
10.29530930519104


In [157]:
technical_features = technical_features.drop(["BP", "TR"], axis = 1)

In [245]:
gains = prices["daily_return"].copy()
losses = prices["daily_return"].copy()

gains.iloc[np.where(prices["daily_return"] <= 0)[0]] = 0
losses.iloc[np.where(prices["daily_return"] >= 0)[0]] = 0
losses = np.abs(losses)
for window in [7, 14, 28, 56, 112]:
    start_time = time.time()
    to_append = np.array([])
    
    for i in Tickers:
        
        RS = gains.loc[i].ewm(span = window).mean() / losses.loc[i].ewm(span = window).mean()
        RSI = 100 - (100 / (1+RS))
        
        to_append = np.append(to_append, RSI)
        
    technical_features["RSI_" + str(window)] = to_append
        
    print(time.time() - start_time)

10.283307790756226
10.16628122329712
10.051255226135254
9.929227590560913
9.912223815917969


In [256]:
technical_features.columns.values

array(['Date', 'Close', 'ABAND_indicator_10', 'ABAND_indicator_20',
       'ABAND_indicator_40', 'ABAND_signal_10', 'ABAND_signal_20',
       'ABAND_signal_40', 'MFV_40_10_indicator_sign_change', 'APO_10_20',
       'APO_10_40', 'AROON_10', 'AROON_20', 'AROON_40', 'AROON_80',
       'AROON_120', 'AROON_160', 'BOL_indicator_10', 'BOL_indicator_20',
       'BOL_indicator_40', 'BOL_indicator_80', 'BOL_indicator_120',
       'BOL_indicator_160', 'CMO_10', 'CMO_20', 'CMO_40', 'CMO_80',
       'CMO_120', 'CMO_160', 'ADX_1_5', 'ADX_1_11', 'ADX_1_23', 'ADX_4_5',
       'ADX_4_11', 'ADX_4_23', 'ADX_16_5', 'ADX_16_11', 'ADX_16_23',
       'DEMA_10', 'DEMA_20', 'DEMA_40', 'DEMA_80', 'DEMA_120', 'DEMA_160',
       'ATR_10', 'ATR_20', 'ATR_40', 'ATR_80', 'ATR_160', 'MFI_10',
       'MFI_20', 'MFI_40', 'MFI_80', 'MFI_160', 'OBV_10', 'OBV_20',
       'OBV_40', 'OBV_80', 'OBV_160', 'close_0/Close_10',
       'close_0/Close_20', 'close_0/Close_40', 'close_0/Close_70',
       'close_0/Close_100', 'close